In [ ]:
import json
import numpy as np
import pandas as pd
import math
import re

## Готовим данные

In [ ]:
fields = [
    l for l in open('data_description.txt', 'r').read().split('\n')
    if len(l.strip()) > 0
]

In [ ]:
columns = [f.strip().split(': ') for f in fields if not f.startswith(' ')]
col_names = {c[0]: f"{c[0]}: {c[1]}" for c in columns}
col_names['SalePrice'] = 'SalePrice'

In [ ]:
last_key = None
fv = {}

for f in fields:
    if not f.startswith(' '):
        last_key = f.split(': ')[0]
        continue
    value_pair = f.strip().split('\t')
    fv[(last_key, value_pair[0])] = value_pair[1]

In [ ]:
s = pd.read_csv('train.csv')

In [ ]:
row_dict = s.iloc[0]

facts = []
for k in row_dict.keys():
    if k not in col_names:
        continue
    if str(row_dict[k]) == 'nan':
        continue
    facts.append(col_names[k])
    if (k, str(row_dict[k])) in fv:
        facts.append(f'- {fv[(k, str(row_dict[k]))]}')
    else:
        facts.append(f'- {row_dict[k]}')
request = '\n'.join(facts).replace('\n- ', ': ')
request = re.sub('\n[^:]+: (?:Identifies )?', '\n', request)
request = re.sub('\n(\d+)', '\nPrice: $\\1', request)

In [ ]:
print(request)

## Делаем запросы в АПИ

In [ ]:
import requests

In [ ]:
result = requests.post(
    url='https://llm.api.cloud.yandex.net/llm/v1alpha/instruct',
    headers={'Authorization': f'Bearer {iamtoken}', 'x-folder-id': dir_id},
    json={
      "model": "general",
      "instruction_text": "Ниже перечислены характеристики недвижимости.",
      "request_text": f"{request}\n\nНа основании этой информации о недвижимости ниже напиши продающий текст на русском. Текст не должен содержать лжи и полностью соответствовать данным из характеристик.",
      "generation_options": {
        "max_tokens": 1500,  
        "temperature": 0.5
      }
    }
)

In [ ]:
print(request)

In [ ]:
print(result.json()['result']['alternatives'][0]['text'])

## Fewshot

In [ ]:
import pickle
train_dataset = pickle.load(open('fewshot.pkl', 'rb'))

In [ ]:
print("\n------\n\n".join(v['request'] + "\n-----\n" + v['response'] for v in train_dataset))

## Finetune

In [ ]:
df = pd.read_json('../squad.json', lines=True)

In [ ]:
df

In [ ]:
df['request'] = df.context + '\n\nQuestion: ' + df.question

In [ ]:
df['response'] = df.ans_texts.str[0]

In [ ]:
df = df.sample(df.shape[0])

In [ ]:
data = [{'request': t.request, 'response': t.response} for t in df[['request', 'response']].itertuples()]

In [ ]:
train = data[:400]

In [ ]:
train = json.load(open('train.json', 'r'))

In [ ]:
sample = df[~df.request.isin([t['request'] for t in train])].sample().iloc[0]
print(sample['request'])
print('-----')
print(sample['response'])